In [ ]:
!pip install langchain langchain_openai langchain_community pypdf faiss-cpu

In [ ]:
from google.colab import drive
import os

# 먼저 구글 드라이브 마운트
drive.mount('/content/drive')

In [1]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv()

# 환경 변수에서 API 키 가져오기
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

file_path = (
    "./Data/투자설명서.pdf"
)
loader = PyPDFLoader(file_path)

doc_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap = 100)

docs = loader.load_and_split(doc_splitter)

In [3]:
from langchain_openai.embeddings import OpenAIEmbeddings

# 데이터를 임베딩으로 변환
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

In [4]:
# FAISS 라이브러리 임포트
from langchain_community.vectorstores import FAISS

# FAISS 벡터스토어 생성
faiss_store = FAISS.from_documents(docs, embedding)
# FAISS 벡터스토어 저장
persist_directory = "./DB"
faiss_store.save_local(persist_directory)

In [5]:
# 저장한 FAISS DB 불러오기
vectordb = FAISS.load_local(persist_directory, embeddings=embedding, allow_dangerous_deserialization=True)

In [6]:
from pydantic import BaseModel, Field
from langchain import PromptTemplate
from langchain.docstore.document import Document
from typing import List, Dict, Any, Tuple
from langchain_openai import ChatOpenAI
from textwrap import dedent
from langchain_core.output_parsers import JsonOutputParser

In [7]:
class RelevanceScore(BaseModel):
    relevance_score: float = Field(description="문서가 쿼리와 얼마나 관련이 있는지를 나타내는 점수.")

def reranking_documents(query: str, docs: List[Document], top_n: int = 2) -> List[Document]:
    parser = JsonOutputParser(pydantic_object=RelevanceScore)
    human_message_prompt = PromptTemplate(
        template = """
        1점부터 10점까지 점수를 매겨, 다음 문서가 질문이 얼마나 관련이 있는지 평가해주세요. 단순히 키워드가 일치하는 것이 아니라 쿼리의 구체적인 맥락과 의도를 고려하세요.
        {format_instructions}
        question: {query}
        document: {doc}
        relevance_score:""",
        input_variables=["query", "doc"],
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )

    llm = ChatOpenAI(temperature=0, model_name="gpt-4o", max_tokens=3000)
    chain = human_message_prompt | llm | parser
    scored_docs = []
    for doc in docs:
        input_data = {"query": query, "doc": doc.page_content}
        try:
            score = chain.invoke(input_data)['relevance_score']
            score = float(score)
        except Exception as e:
            print(f"오류 발생: {str(e)}")
            default_score = 5  # 기본 점수를 5점으로 설정
            print(f"기본 점수 {default_score}점을 사용합니다.")
            score = default_score
        scored_docs.append((doc, score))

    reranked_docs = sorted(scored_docs, key=lambda x: x[1], reverse=True)
    return [doc for doc, _ in reranked_docs[:top_n]]

In [8]:
query = "이 회사의 2022년 영업손실이 정확히 얼마야?"
initial_docs = vectordb.similarity_search(query, k=4)
reranked_docs = reranking_documents(query, initial_docs)

In [9]:
# print first 4 initial documents
print(f"Query: {query}\n\n")

print("Top initial documents:")
for i, doc in enumerate(initial_docs):
    print(f"\nDocument {i+1}:")
    print(doc.page_content)


# Print results
print("\n\nTop reranked documents:")
for i, doc in enumerate(reranked_docs):
    print(f"\nDocument {i+1}:")
    print(doc.page_content)  # Print first 200 characters of each document

Query: 이 회사의 2022년 영업손실이 정확히 얼마야?


Top initial documents:

Document 1:
이 주요 원인으로 작용하여 당기순손실 228.7억원이 발생하였습니다. 
 
이어, 당사는 2023년에는 매출이 발생하지 않았으며, 종업원급여 40.9억원, 유무형자산상각
비 26.3억원이 발생하였으며, 연구개발 및 임상시험 지속 과정에서 지급수수료가 30.5억원
발생하는 등 총 영업비용 122억원이 발생하였습니다. 이에 2023년 영업손실 122억원이 발
생하였으며, 금융손익 및 기타손익 반영 후 당기순손실 116.1억원이 발생하였습니다. 한편,

Document 2:
이어, 당사는 2022년 GMP시설의 위탁생산계약 매출 약 4.8억원이 발생하였으나, 종업원급
여 43.9억원, 유무형자산상각비 25.3억원이 발생하였으며, 연구개발 및 임상시험 지속 과정
에서 지급수수료가 54.8억원 발생하는 등 총 영업비용 154억원이 발생하였습니다. 이에
2022년 영업손실 149억원이 발생하였으며, 한편, 2022년 5월 금융감독원에서 공표한 '전환
사채 콜옵션 회계처리' 감독지침에 따라 당사는 2021년 3월 19일 발행한 제2회 전환사채의

Document 3:
하여 2021년 영업손실 130.1억원, 2022년 영업손실 149.1억원, 2023년 영업손실 122억원, 2024년
1분기 영업손실 24.2억원이 발생하였습니다. 또한 영업 외적 측면에서도, 금융비용 등의 발생 영향
으로 인해 2021년 당기순손실 130.7억원, 2022년 당기순손실 228.7억원, 2023년 당기순손실
116.1억원, 2024년 1분기 당기순손실 32.9억원이 발생하는 등 지속적인 적자 구조를 면하지 못하고
있습니다.따라서 당사의 파이프라인에서 임상 성공을 통한 기술이전, 상품화 성공 등의 성과를 이루

Document 4:
2020년 11월 GMP 완공에 따라 2021년에는 소모품비 21.8억원, 유무형자산상각비 22.4억
원이 발생하였으며, 코로나19 예방

In [10]:
from langchain_core.retrievers import BaseRetriever
from langchain.chains import RetrievalQA

In [11]:
# 커스텀 리트리버 체인을 생성합니다.
class CustomRetriever(BaseRetriever):

    vectorstore: Any = Field(description="Retrival을 위한 벡터스토어")

    class Config:
        arbitrary_types_allowed = True

    # num_docs 파라미터로 리랭킹 후 반환할 최종 문서의 수를 정의합니다.
    def get_relevant_documents(self, query: str, num_docs=2) -> List[Document]:
        initial_docs = self.vectorstore.similarity_search(query, k=4)
        return reranking_documents(query, initial_docs, top_n=num_docs)

C:\Users\User\AppData\Local\Temp\ipykernel_19652\4046516056.py:2: DeprecationWarning: Retrievers must implement abstract `_get_relevant_documents` method instead of `get_relevant_documents`
  class CustomRetriever(BaseRetriever):


In [12]:
# custom retriever 인스턴스를 생성합니다.
custom_retriever = CustomRetriever(vectorstore=vectordb)

# 답변용 LLM 인스턴스를 생성합니다.
llm = ChatOpenAI(temperature=0.2, model_name="gpt-4o")

# RetrievalQA 체인을 생성합니다.
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=custom_retriever,
    return_source_documents=True
)

In [13]:
qa_chain.invoke("이 회사의 2022년 영업손실이 정확히 얼마야?")

{'query': '이 회사의 2022년 영업손실이 정확히 얼마야?',
 'result': '2022년 영업손실은 149.1억원입니다.',
 'source_documents': [Document(id='a5a19932-03e6-44af-8ca5-cbb368370c9b', metadata={'producer': 'iText® 5.5.9 ©2000-2015 iText Group NV (AGPL-version)', 'creator': 'PyPDF', 'creationdate': '2024-06-26T16:15:14+09:00', 'moddate': '2024-06-26T16:15:14+09:00', 'source': './Data/투자설명서.pdf', 'total_pages': 514, 'page': 158, 'page_label': '159'}, page_content='하여 2021년 영업손실 130.1억원, 2022년 영업손실 149.1억원, 2023년 영업손실 122억원, 2024년\n1분기 영업손실 24.2억원이 발생하였습니다. 또한 영업 외적 측면에서도, 금융비용 등의 발생 영향\n으로 인해 2021년 당기순손실 130.7억원, 2022년 당기순손실 228.7억원, 2023년 당기순손실\n116.1억원, 2024년 1분기 당기순손실 32.9억원이 발생하는 등 지속적인 적자 구조를 면하지 못하고\n있습니다.따라서 당사의 파이프라인에서 임상 성공을 통한 기술이전, 상품화 성공 등의 성과를 이루'),
  Document(id='0448fb1b-066d-407d-9dd6-958fa171ae76', metadata={'producer': 'iText® 5.5.9 ©2000-2015 iText Group NV (AGPL-version)', 'creator': 'PyPDF', 'creationdate': '2024-06-26T16:15:14+09:00', 'moddate': '2024-06-26T16:15:14+09:00', 'source': './Data/투자설명서